In [ ]:
import os
import numpy as np
from glob import glob
import tqdm
import shutil
import json
import copy
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import torchvision.transforms as T
import torch
from torch.nn.functional import pad

In [ ]:
LABEL_PATH = "gtFine_trainvaltest/gtFine"
IMG_PATH = "leftImg8bit_trainvaltest/leftImg8bit"
BBOX_PATH = "gtBbox_cityPersons_trainval/gtBboxCityPersons"
ONLY_HUMAN_PATH = "only_human"
FINAL_IMG_PATH = "final_data"
PED_ID = 24
RIDER_ID = 25
IMAGE_SIZE = 256
MIN_PIXEL_HEIGHT = 99

In [ ]:
labels = sorted(glob(os.path.join(LABEL_PATH, "*/*/*_labelIds.png")))

In [ ]:
def filter_only_humans(labels):
    # keep only the labels with pedestrians in them
    labels_with_human = []
    for label in tqdm.tqdm(labels):
        mask = cv2.imread(label, cv2.IMREAD_GRAYSCALE)
        if PED_ID in np.unique(mask) or RIDER_ID in np.unique(mask):
            labels_with_human.append(label)
    save_in_folder(ONLY_HUMAN_PATH, labels_with_human)
    
    return labels_with_human

def filter_only_sig_humans(labels):
    # keep only the labels with pedestrians that's bigger than 100 pixels
    removed = []
    for label in labels:
        path = label.split("gtFine\\")[-1]
        path = path.replace("_gtFine_labelIds", "_gtBboxCityPersons")
        path = path.replace(".png", ".json")
        path = os.path.join(BBOX_PATH, path)
        path = os.path.join(ONLY_HUMAN_PATH, path)
        with open(path) as f:
            data = json.load(f)
    
        if all(obj['label'] == 'ignore' or obj['bbox'][-1] < 100 for obj in data['objects']):
            removed.append(label)
    
    for file in removed:
        label = file
        os.remove(label)

        key = label.split("gtFine\\")[-1]
        img = key.replace("_gtFine_labelIds", "_leftImg8bit")
        img = os.path.join(IMG_PATH, img)
        img = os.path.join(ONLY_HUMAN_PATH, img)
        os.remove(img)

        bbox = key.replace("_gtFine_labelIds", "_gtBboxCityPersons")
        bbox = bbox.replace(".png", ".json")
        bbox = os.path.join(BBOX_PATH, bbox)
        bbox = os.path.join(ONLY_HUMAN_PATH, bbox)
        os.remove(bbox)
    return [l for l in labels if l not in removed]

def save_in_folder(folder_name, labels):
    for label in tqdm.tqdm(labels):
        # copy label over
        src = label
        
        dst = os.path.join(folder_name, src)
        dstfolder = os.path.dirname(dst)
        
        if not os.path.exists(dstfolder):
            os.makedirs(dstfolder)
        shutil.copy(src,dst)
    
        # copy image over
        src = src.split("gtFine\\")[-1]
        src = src.replace("_gtFine_labelIds", "_leftImg8bit")
        src = os.path.join(IMG_PATH, src)
        
        dst = os.path.join(folder_name, src)
        dstfolder = os.path.dirname(dst)
        
        if not os.path.exists(dstfolder):
            os.makedirs(dstfolder)
        shutil.copy(src,dst)
        
        # copy bbox over
        src = src.split("gtFine\\")[-1]
        src = src.replace("_gtFine_labelIds", "_gtBboxCityPersons")
        src = src.replace(".png", ".json")
        src = os.path.join(BBOX_PATH, src)

        dst = os.path.join(folder_name, src)
        dstfolder = os.path.dirname(dst)

        if not os.path.exists(dstfolder):
            os.makedirs(dstfolder)
        shutil.copy(src,dst)

In [ ]:
# labels_with_human = filter_only_humans(labels)

path = os.path.join(LABEL_PATH, "*/*/*_labelIds.png")
path = os.path.join(ONLY_HUMAN_PATH, path)
labels_with_human = sorted(glob(path))

In [ ]:
# No pedestrian or rider labels in TEST dataset
for x in labels_with_human:
    assert("test\\" not in x)

In [ ]:
len(labels_with_human)

In [ ]:
# Filter out masks with puny humans 
labels_with_human = filter_only_sig_humans(labels_with_human)

In [ ]:
# combine intersecting bounding boxes together
def combineBoundingBox(box1, box2):
    if box1[0] <= box2[0] and box1[1] <= box2[1]:
        box_out = box1
        box_in = box2
    else:
        box_out = box2
        box_in = box1
        
    x1,y1,w1,h1 = box_in
    x2,y2,w2,h2 = box_out
    x = min(x1, x2)
    y = min(y1, y2)
    w = max(x2+w2, x1+w1) - x
    h = max(y2+h2, y1+h1) - y
    return [x, y, w, h]

def bb_intersection_over_union(boxA, boxB):
    x1,y1,w1,h1 = boxA
    x2,y2,w2,h2 = boxB
    xA = max(x1, x2)
    yA = max(y1, y2)
    xB = min(x1+w1, x2+w2)
    yB = min(y1+h1, y2+h2)
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = w1*h1
    boxBArea = w2*h2
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def combine(bboxes):
    passed = False
    while(not passed):
        passed = True
        for bbox_id1, bbox1 in bboxes.items():
            for bbox_id2, bbox2 in bboxes.items():
                if bbox_id1 == bbox_id2: continue
                if bb_intersection_over_union(bbox1, bbox2) > 0:
                    new_bbox = combineBoundingBox(bbox1, bbox2)
                    bboxes.pop(bbox_id2)
                    bboxes[bbox_id1] = new_bbox
                    passed = False
                    break
            if not passed: 
                break 
    return bboxes

In [ ]:
# create new image patches
for label in tqdm.tqdm(labels_with_human):
    key = label.split("gtFine\\")[-1]
    
    # mask
    mask = cv2.imread(label, cv2.IMREAD_GRAYSCALE)
    
    # img
    path = key.replace("_gtFine_labelIds", "_leftImg8bit")
    path = os.path.join(IMG_PATH, path)
    path = os.path.join(ONLY_HUMAN_PATH, path)
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    
    # only human mask
#     orig_img = copy.deepcopy(img)
#     mask = (mask == PED_ID) | (mask == RIDER_ID)
#     mask = mask.astype(float)
#     h, w = mask.shape
#     for row in range(h):
#         for col in range(w):
#             if not mask[row][col]:
#                 img[row][col] = [255, 255, 255]

    #get bounding boxes
    path = key.replace("_gtFine_labelIds", "_gtBboxCityPersons")
    path = path.replace(".png", ".json")
    path = os.path.join(BBOX_PATH, path)
    path = os.path.join(ONLY_HUMAN_PATH, path)
    with open(path) as f:
        data = json.load(f)
        
    # filter and combine bounding boxes
    bboxes = {obj['instanceId']:obj['bbox'] for obj in data['objects'] 
          if obj['label'] != 'ignore' and 
          obj['bbox'][0] > 0 and 
          obj['bbox'][1] > 0 and
          obj['bbox'][-1] > MIN_PIXEL_HEIGHT}
    bboxes = combine(bboxes)
    
    # create square bounding box
    for bbox_id, (x,y,w,h) in bboxes.items():
        
        # create 256 by 256 image patch for small bboxes
        if w < IMAGE_SIZE and h < IMAGE_SIZE:
            x = x - np.random.randint(0, min(x, IMAGE_SIZE - w))
            y = y - np.random.randint(0, min(y, IMAGE_SIZE - h))
            w = IMAGE_SIZE
            h = IMAGE_SIZE
        else:
            # add padding
            x = x - min(x, 10)
            y = y - min(y, 10)
            w = w + min(x, 10)
            h = h + min(y, 10)
            w = w + min(2048-(x+w), 10)
            h = h + min(1024-(y+h), 10)
    
            if w > h:
                y = y - (w - h) // 2
                h = h + (w - h)
            else:
                x = x - (h - w) // 2
                w = w + (h - w)
            
#         img = cv2.rectangle(img, (x,y), (x+w, y+h), color=(255,0,0), thickness=3) 
        bboxes[bbox_id] = [x,y,w,h]
    
    # save cooresponding image patches and mask patches
    for bbox_id, (x,y,w,h) in bboxes.items():
        img_tmp = img[y:y+h, x:x+w]
        mask_tmp = mask[y:y+h, x:x+w]

        # save img
        path = key.replace("gtFine_labelIds", str(bbox_id)+"_leftImg8bit")
        path = os.path.join(IMG_PATH, path)
        path = os.path.join(FINAL_IMG_PATH, path)
        dstfolder = os.path.dirname(path)
        if not os.path.exists(dstfolder):
            os.makedirs(dstfolder)  
        cv2.imwrite(path, img_tmp)

        # save mask
        path = key.replace("labelIds", str(bbox_id)+"_labelIds")
        path = os.path.join(LABEL_PATH, path)
        path = os.path.join(FINAL_IMG_PATH, path)
        dstfolder = os.path.dirname(path)
        if not os.path.exists(dstfolder):
            os.makedirs(dstfolder)  
        cv2.imwrite(path, mask_tmp)

In [ ]:
# sighh..there are 70 corrupted files...remove them
corrupt_files = []
files = glob(os.path.join(FINAL_IMG_PATH, "*/*/*/*/*_leftImg8bit.png"))
for file in tqdm.tqdm(files):
    try:
        cv2.cvtColor(cv2.imread(file, cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB) 
    except:
        corrupt_files.append(file)

In [ ]:
for file in corrupt_files:
    os.remove(file)
    parts = file.split("_")
    file = "_".join(parts[:-2]) + "_gtFine_" + parts[-2] + "_labelIds.png"
    file = file.replace("leftImg8bit_trainvaltest\\leftImg8bit", "gtFine_trainvaltest\\gtFine")
    os.remove(file)